# Title
[]()

In [1]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
from silvhua import *

from openai import OpenAI
import os
from time import time

from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.agents import Tool


In [2]:

conversation_dict = dict()
system_message_dict = dict()
reply_dict = dict()
contacts = load_json('contacts.json', '../src/app/private')


# Load `inbound_dict`

In [8]:
inbound_filename = '/home/silvhua/repositories/GHL-chat/events/InboundMessage_content.json'

inbound_dict = load_json(inbound_filename, '')
inbound_dict


{'alert': {'1': 'Can you give me an Olympic lifting program to improve my technique?',
  '2': 'I have a diagnosed eating disorder. Can you help me with that?',
  '3': 'What is the capital of Canada?',
  '4': "Yes, let's book a call"},
 'no_alert': {'1': 'What does the program involve?',
  '2': 'What are your qualifications?',
  '3': 'I feel lost with my nutrition',
  '4': 'Is this a bot or human?'}}

# Iteration 1

In [3]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Chatbot_Response(BaseModel):
    response: str = Field(description="The response to the InboundMessage.")
    alert_human: bool = Field(description="Whether or not to alert a human to review the response.")

def create_system_message(
        business_name, 
        prompts_filepath='../private/prompts',
        examples_filepath='../private/data/chat_examples', doc_filepath='../private/data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

## Other Messages

Only repond to inbound messages that can be answered by the message templates or provided 
documenation. Otherwise, return "[ALERT HUMAN]". 
The "[ALERT HUMAN]" message will trigger a human staff member to review the messages to write a response. 
It is better to err on the side of caution and flag a staff rather than give a wrong response.

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the message templates or documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
            "input": user_input,
            "chat_history": chat_history
        })  
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def alert_human(str) -> str:
    return {"response": "[ALERT HUMAN]", "alert_human": True}

def placeholder_function(str):
    return None

tools = [
    # Tool(
    #     name=f"alert_human_function",
    #     func=alert_human,
    #     description=f"If the InboundMessage cannot be answered from your instructions or documentation, use this function.",
    # ),
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"This function does not do anything.",
    )
]

conversation_id = 1
InboundMessage = "Tell me more about the program"
reply_dict[conversation_id] = dict()
question_id = 1

contactId = 'test_cm'
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', 
    prompts_filepath='../private/prompts',
    examples_filepath='../private/data/chat_examples', doc_filepath='../private/data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


Chat history length: 0


> Entering new AgentExecutor chain...
{
  "response": "I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. The Strong and Sassy programme includes 1:1 nutrition habit coaching, personalized resistance training programs, weekly group coaching calls, in-house mindset coaching, nutrition education, unlimited accountability, community meet-ups, and free recipe e-books.",
  "alert_human": false
}
{
  "response": "I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. The Strong and Sassy programme includes 1:1 nutrition habit coaching, personalized resistance training programs, weekly group coaching calls, in-house mindset coaching, nutrition education, unlimited accountab

In [14]:
question_id = 2
alert = 'no_alert'
InboundMessage = inbound_dict[alert][str(question_id)]


reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

Chat history length: 0


> Entering new AgentExecutor chain...
{"response":"I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.","alert_human":false}
{"response":"I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.","alert_human":false}

> Finished chain.
Response time: 3.3333845138549805 seconds
Reply from `chat_with_chatbot`: {"response":"I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.","alert_human":false}
{"response":"I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of

In [13]:
inbound_dict[alert]['2']

'What are your qualifications?'

# Iteration 2

In [15]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Chatbot_Response(BaseModel):
    response: str = Field(description="The response to the InboundMessage.")
    alert_human: bool = Field(description="Whether or not to alert a human to review the response.")

def create_system_message(
        business_name, 
        prompts_filepath='../private/prompts',
        examples_filepath='../private/data/chat_examples', doc_filepath='../private/data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""{instructions}

## Other Messages

Only repond to inbound messages that can be answered by the message templates or provided 
documenation. Otherwise, return "[ALERT HUMAN]". 
The "[ALERT HUMAN]" message will trigger a human staff member to review the messages to write a response. 
It is better to err on the side of caution and flag a staff rather than give a wrong response.
    
# Stage 1

Determine if you should generate a response to the inbound message. If so, generate the response and proceed 
to Stage 2. Otherwise, return "[ALERT HUMAN]".

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the message templates or documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to revise as needed to make it concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
            "input": user_input,
            "chat_history": chat_history
        })  
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def alert_human(str) -> str:
    return {"response": "[ALERT HUMAN]", "alert_human": True}

def placeholder_function(str):
    return None

tools = [
    # Tool(
    #     name=f"alert_human_function",
    #     func=alert_human,
    #     description=f"If the InboundMessage cannot be answered from your instructions or documentation, use this function.",
    # ),
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"This function does not do anything.",
    )
]

conversation_id = 2
reply_dict[conversation_id] = dict()
question_id = 1
InboundMessage = inbound_dict[alert][str(question_id)]

contactId = 'test_cm'
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', 
    prompts_filepath='../private/prompts',
    examples_filepath='../private/data/chat_examples', doc_filepath='../private/data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


Chat history length: 0


> Entering new AgentExecutor chain...
{"response":"I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves. The Strong and Sassy programme includes 1:1 Nutrition habit coaching, Personalised Resistance training programmes, Weekly group coaching calls, In House mindset coach, Nutrition education, Unlimited Accountability, Community meet ups, and Free Recipe E-Books.","alert_human":false}
{"response":"I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, s

In [16]:
question_id +=1
alert = 'no_alert'
InboundMessage = inbound_dict[alert][str(question_id)]
print(f'InboundMessage: {InboundMessage}\n')

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

InboundMessage: What are your qualifications?

Chat history length: 0


> Entering new AgentExecutor chain...
{"response":"I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.","alert_human":false}
{"response":"I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.","alert_human":false}

> Finished chain.
Response time: 3.204585075378418 seconds
Reply from `chat_with_chatbot`: {"response":"I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.","alert_human":false}
{"response":"I am a degree qualified nutritionist and coach with over 10 years exper

In [17]:
reply_dict[conversation_id][question_id]['intermediate_steps']

[]

In [18]:
question_id +=1
alert = 'no_alert'
InboundMessage = inbound_dict[alert][str(question_id)]
print(f'InboundMessage: {InboundMessage}\n')

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

InboundMessage: I feel lost with my nutrition

Chat history length: 0


> Entering new AgentExecutor chain...

		{
	"response": "Ah that's a great goal! Just curious, why is <the desired result> important to you, and how will that impact your life? I'd love to help you out the best that I can! ❤️",
	"alert_human": false
}

> Finished chain.
Response time: 2.094564437866211 seconds
Reply from `chat_with_chatbot`: 
		{
	"response": "Ah that's a great goal! Just curious, why is <the desired result> important to you, and how will that impact your life? I'd love to help you out the best that I can! ❤️",
	"alert_human": false
}


In [19]:
question_id +=1
alert = 'no_alert'
InboundMessage = inbound_dict[alert][str(question_id)]
print(f'InboundMessage: {InboundMessage}\n')

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

InboundMessage: Is this a bot or human?

Chat history length: 0


> Entering new AgentExecutor chain...

    {
        "response": "[ALERT HUMAN]",
        "alert_human": true
    }

    {
        "response": "[ALERT HUMAN]",
        "alert_human": true
    }

> Finished chain.
Response time: 1.930250644683838 seconds
Reply from `chat_with_chatbot`: 
    {
        "response": "[ALERT HUMAN]",
        "alert_human": true
    }

    {
        "response": "[ALERT HUMAN]",
        "alert_human": true
    }


In [20]:
question_id = 1
alert = 'alert'
InboundMessage = inbound_dict[alert][str(question_id)]
print(f'InboundMessage: {InboundMessage}\n')

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

InboundMessage: Can you give me an Olympic lifting program to improve my technique?

Chat history length: 0


> Entering new AgentExecutor chain...

  {"response": "[ALERT HUMAN]", "alert_human": true}

> Finished chain.
Response time: 1.186612606048584 seconds
Reply from `chat_with_chatbot`: 
  {"response": "[ALERT HUMAN]", "alert_human": true}


In [21]:
question_id += 1
alert = 'alert'
InboundMessage = inbound_dict[alert][str(question_id)]
print(f'InboundMessage: {InboundMessage}\n')

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

InboundMessage: I have a diagnosed eating disorder. Can you help me with that?

Chat history length: 0


> Entering new AgentExecutor chain...

    {"response": "[ALERT HUMAN]", "alert_human": true}

> Finished chain.
Response time: 1.927816390991211 seconds
Reply from `chat_with_chatbot`: 
    {"response": "[ALERT HUMAN]", "alert_human": true}


In [22]:
question_id += 1
alert = 'alert'
InboundMessage = inbound_dict[alert][str(question_id)]
print(f'InboundMessage: {InboundMessage}\n')

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

InboundMessage: What is the capital of Canada?

Chat history length: 0


> Entering new AgentExecutor chain...

  {
    "response": "[ALERT HUMAN]",
    "alert_human": true
  }

> Finished chain.
Response time: 1.2443015575408936 seconds
Reply from `chat_with_chatbot`: 
  {
    "response": "[ALERT HUMAN]",
    "alert_human": true
  }


In [23]:
reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

Chat history length: 0


> Entering new AgentExecutor chain...

  {
    "response": "[ALERT HUMAN]",
    "alert_human": true
  }

> Finished chain.
Response time: 1.3051986694335938 seconds
Reply from `chat_with_chatbot`: 
  {
    "response": "[ALERT HUMAN]",
    "alert_human": true
  }


In [24]:
reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

Chat history length: 0


> Entering new AgentExecutor chain...
  {"response":"[ALERT HUMAN]","alert_human":true}
{"response":"[ALERT HUMAN]","alert_human":true}

> Finished chain.
Response time: 1.6409404277801514 seconds
Reply from `chat_with_chatbot`:   {"response":"[ALERT HUMAN]","alert_human":true}
{"response":"[ALERT HUMAN]","alert_human":true}


In [25]:
question_id += 1
alert = 'alert'
InboundMessage = inbound_dict[alert][str(question_id)]
print(f'InboundMessage: {InboundMessage}\n')

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

InboundMessage: Yes, let's book a call

Chat history length: 0


> Entering new AgentExecutor chain...
{
  "response": "[ALERT HUMAN]",
  "alert_human": true
}

> Finished chain.
Response time: 1.2324841022491455 seconds
Reply from `chat_with_chatbot`: {
  "response": "[ALERT HUMAN]",
  "alert_human": true
}


# *End of Page*